# Curve Fitting

[What is curve fitting]

## Setup

First, let's import all the modules we'll need. We're going to use Plotly as our graphing library instead of Matplotlib.

Furthermore, we will also make use of sympy's capabilities in this project.

In [21]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [22]:
pd.options.plotting.backend = "plotly"

In [23]:
import sympy
x_sub = sympy.Symbol("x")

## Inputs

In here, you can adjust the given data that will be used in Polynomial Regression and Newton's Interpolation. Try to keep the data small (n <= 10) so that performance won't be affected as much.

In [24]:
data = pd.DataFrame({
    'x': np.array([2.5, 3.5, 5, 6, 7.5, 10, 12.5, 15, 17.5, 20]),
    'y': np.array([7, 5.5, 3.9, 3.6, 3.1, 2.8, 2.6, 2.4, 2.3, 2.1]),
})

data.head()

,x,y
0,2.5,7.0
1,3.5,5.5
2,5.0,3.9
3,6.0,3.6
4,7.5,3.1


Below is a scatterplot of the inputs shown above. We're going to try to find an equation that both best fits the points below and one that actually fits all the points below.

In [25]:
data.plot.scatter(x="x", y="y", title="Given Data")

In [26]:
leftmost = min(data['x'])
rightmost = max(data['x'])

## Regression Analysis

In [27]:
 # We need to specify
n = 3

In [28]:
regression = {
    'y': data['y'].to_numpy(),
    'x': data['x'].to_numpy(),
}


regression

{'y': array([7. , 5.5, 3.9, 3.6, 3.1, 2.8, 2.6, 2.4, 2.3, 2.1]),
 'x': array([ 2.5,  3.5,  5. ,  6. ,  7.5, 10. , 12.5, 15. , 17.5, 20. ])}

In [29]:
last_exponent = (n - 1)*2

for i in range(last_exponent + 1):
    regression[f'x**{i}'] = sum(regression['x'] ** i)

for i in range(n):
    regression[f'x**{i}*y'] = sum(regression['x'] ** i * regression['y'])

regression

{'y': array([7. , 5.5, 3.9, 3.6, 3.1, 2.8, 2.6, 2.4, 2.3, 2.1]),
 'x': array([ 2.5,  3.5,  5. ,  6. ,  7.5, 10. , 12.5, 15. , 17.5, 20. ]),
 'x**0': 10.0,
 'x**1': 99.5,
 'x**2': 1323.25,
 'x**3': 20508.875,
 'x**4': 344102.3125,
 'x**0*y': 35.300000000000004,
 'x**1*y': 279.85,
 'x**2*y': 3283.225}

In [30]:
y_matrix = []
x_matrix = []

for i in range(n):
    x_matrix.append([
                     regression[f'x**{i}'],
                     regression[f'x**{i+1}'],
                     regression[f'x**{i+2}'],
    ])
    y_matrix.append([regression[f'x**{i}*y']])

x_matrix = np.matrix(x_matrix)
y_matrix = np.matrix(y_matrix)

x_matrix, y_matrix

(matrix([[1.00000000e+01, 9.95000000e+01, 1.32325000e+03],
         [9.95000000e+01, 1.32325000e+03, 2.05088750e+04],
         [1.32325000e+03, 2.05088750e+04, 3.44102312e+05]]),
 matrix([[  35.3  ],
         [ 279.85 ],
         [3283.225]]))

In [31]:
coefficients = np.linalg.inv(x_matrix) @ y_matrix
coefficients = np.squeeze(np.asarray(coefficients))
coefficients

array([ 7.89243713, -0.77976602,  0.02566586])

In [32]:
fn = [item*x_sub**i for i, item in enumerate(coefficients)]
fn = sum(fn)
fn

0.0256658591674564*x**2 - 0.779766024178166*x + 7.89243712623906

In [33]:
fn = sympy.lambdify('x', fn)

In [34]:
plot_data_regression = pd.DataFrame([], columns=['x', 'regression'])
plot_data_regression['x'] = np.arange(leftmost, rightmost, 0.01)
plot_data_regression['regression'] = fn(plot_data_regression['x'])

plot_data_regression['x_merge'] = plot_data_regression['x'] * 1e4
data['x_merge'] = data['x'] * 1e4

plot_data_regression = pd.merge(plot_data_regression, data, on="x_merge", how="outer")
plot_data_regression = plot_data_regression[['x_x', 'x_y', 'regression', 'y']]

plot_data_regression.head()

,x_x,x_y,regression,y
0,2.50,2.5,6.103434,7.0
1,2.51,NaN,6.096922,NaN
2,2.52,NaN,6.090415,NaN
3,2.53,NaN,6.083914,NaN
4,2.54,NaN,6.077417,NaN


In [35]:
plot1 = go.Scatter(
    x=plot_data_regression["x_x"],
    y=plot_data_regression["regression"],
    mode='lines',
    line_color='blue',
    name="Regression",
)
plot2 = go.Scatter(
    x=plot_data_regression['x_y'],
    y=plot_data_regression["y"],
    mode='markers',
    line=dict(color="red"),
    name="Actual",
)

fig = go.Figure(data=plot1)
fig.add_trace(plot2)

fig.update_layout(
    title="Polynomial Regression",
    xaxis_title="x",
    yaxis_title="y",
)

fig.show()

## Newton Interpolation

In [36]:
def delta_finder(data, index):
    inside_data = data.copy()
    data_length = len(inside_data)

    for item in inside_data.index:
        col = f'delta{index - 1}' if index > 1 else 'y'
        
        try:
            y2 = inside_data.loc[item + 1, col]
            x2 = inside_data.loc[item + index, 'x']
        except KeyError:
            inside_data.loc[item, f'delta{index}'] = None
            continue
        
        y1 = inside_data.loc[item, col]
        x1 = inside_data.loc[item, 'x']

        inside_data.loc[item, f'delta{index}'] = (y2 - y1) / (x2 - x1)

    return inside_data


for i in range(len(data) - 1):
    data = delta_finder(data, i + 1)

data = data.drop('x_merge', axis=1)
data.head()

,x,y,delta1,delta2,delta3,delta4,delta5,delta6,delta7,delta8,delta9
0,2.5,7.0,-1.500000,0.173333,0.038095,-0.023619,0.005064,-6.960033e-04,0.000071,-0.000006,3.945795e-07
1,3.5,5.5,-1.066667,0.306667,-0.080000,0.014359,-0.001896,1.941484e-04,-0.000016,0.000001,NaN
2,5.0,3.9,-0.300000,-0.013333,0.013333,-0.002708,0.000336,-2.999509e-05,0.000002,NaN,NaN
3,6.0,3.6,-0.333333,0.053333,-0.006974,0.000656,-0.000039,-2.955885e-07,NaN,NaN,NaN
4,7.5,3.1,-0.120000,0.008000,-0.001067,0.000213,-0.000043,NaN,NaN,NaN,NaN


In [37]:
x = data['x']
b = data[data.index == 0].transpose()
b = b[1:].squeeze()

x, b

(0     2.5
 1     3.5
 2     5.0
 3     6.0
 4     7.5
 5    10.0
 6    12.5
 7    15.0
 8    17.5
 9    20.0
 Name: x, dtype: float64,
 y         7.000000e+00
 delta1   -1.500000e+00
 delta2    1.733333e-01
 delta3    3.809524e-02
 delta4   -2.361905e-02
 delta5    5.063736e-03
 delta6   -6.960033e-04
 delta7    7.121213e-05
 delta8   -5.814825e-06
 delta9    3.945795e-07
 Name: 0, dtype: float64)

In [38]:
from functools import reduce

In [39]:
fn = []

for i, val in enumerate(b):
    factors = [(x_sub - x[data.index == j]) for j in range(i)]
    factors = reduce(lambda a, b: a * b, factors, 1)
    fn.append(val*factors)

fn = sum(fn)
fn.simplify()

3.94579518786558e-7*x**9 - 3.71838971757481e-5*x**8 + 0.00149650414039423*x**7 - 0.0336330911322494*x**6 + 0.463188491225361*x**5 - 4.03346428392863*x**4 + 22.0716656547838*x**3 - 72.5292604521313*x**2 + 127.546525551743*x - 83.7512212425256

In [40]:
fn = sympy.lambdify('x', fn)

In [44]:
plot_data_newton = pd.DataFrame([], columns=['x', 'interpolation'])
plot_data_newton['x'] = np.arange(leftmost, rightmost, 0.01)
plot_data_newton['interpolation'] = fn(plot_data_newton['x'])

plot_data_newton['x_merge'] = plot_data_newton['x'] * 1e4
data['x_merge'] = data['x'] * 1e4

plot_data_newton = pd.merge(plot_data_newton, data, on="x_merge", how="outer")
plot_data_newton = plot_data_newton[['x_x', 'x_y', 'interpolation', 'y']]

plot_data_newton.head()

,x_x,x_y,interpolation,y
0,2.50,2.5,7.000000,7.0
1,2.51,NaN,6.997660,NaN
2,2.52,NaN,6.994782,NaN
3,2.53,NaN,6.991375,NaN
4,2.54,NaN,6.987453,NaN


In [46]:
plot1 = go.Scatter(
    x=plot_data_newton["x_x"],
    y=plot_data_newton["interpolation"],
    mode='lines',
    line_color='blue',
    name="Interpolation",
)
plot2 = go.Scatter(
    x=plot_data_newton['x_y'],
    y=plot_data_newton["y"],
    mode='markers',
    line=dict(color="red"),
    name="Interpolation",
)

fig = go.Figure(data=plot1)
fig.add_trace(plot2)

fig.update_layout(
    title="Newton Interpolation",
    xaxis_title="x",
    yaxis_title="y",
)

fig.show()

## Comparison and Conclusion

As you can see, Newton Interpolation actually tries to fit all the given data in a single equation while Polynomial Regression tries to find an equation that follows the trends in the data.

In [47]:
regression_data = go.Scatter(
    x=plot_data_regression["x_x"],
    y=plot_data_regression["regression"],
    mode='lines',
    line_color='blue',
    name="Regression",
)
interpolation_data = go.Scatter(
    x=plot_data_newton["x_x"],
    y=plot_data_newton["interpolation"],
    mode='lines',
    line_color='green',
    name="Interpolation",
)
original_data = go.Scatter(
    x=data['x'],
    y=data["y"],
    mode='markers',
    line=dict(color="red"),
    name="Actual",
)

fig = go.Figure(data=regression_data)
fig.add_trace(interpolation_data)
fig.add_trace(original_data)

fig.update_layout(
    title="Comparison of Polynomial Regression and Newton Interpolation",
    xaxis_title="x",
    yaxis_title="y",
)

fig.show()